# Parameter handling
The tool within $C^3$ to manipulate the parameters of both the model and controls is the `ParameterMap`. It provides methods to present the same data for human interaction, i.e. structured information with physical units and for numerical optimization algorithms that prefer a linear vector of scale 1. Here, we'll show some example usage.
We'll use the `ParameterMap` of the model also used in the simulated calibration example.

In [1]:
import sys
sys.path.append('/home/llguo/PycharmProjects/c3/')

In [2]:
from single_qubit_blackbox_exp import create_experiment

exp = create_experiment()
pmap = exp.pmap

2022-06-02 14:46:18.486863: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-02 14:46:18.486893: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-06-02 14:46:20.910069: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2022-06-02 14:46:20.910116: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: SQD
2022-06-02 14:46:20.910125: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: SQD
2022-06-02 14:46:20.910202: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.129.6
2022-06-02 14:46:20.910235: I tensorflow/

The pmap contains a list of all parameters and their values:

In [5]:
pmap.get_full_params()

{'Q1-freq': 5.000 GHz 2pi,
 'Q1-anhar': -210.000 MHz 2pi,
 'Q1-temp': 0.000 K,
 'init_ground-init_temp': -3.469 aK,
 'resp-rise_time': 300.000 ps,
 'v_to_hz-V_to_Hz': 1.000 GHz/V,
 'id[0]-d1-no_drive-amp': 1.000 V,
 'id[0]-d1-no_drive-delta': 0.000 V,
 'id[0]-d1-no_drive-freq_offset': 0.000 Hz 2pi,
 'id[0]-d1-no_drive-xy_angle': 0.000 rad,
 'id[0]-d1-no_drive-sigma': 5.000 ns,
 'id[0]-d1-no_drive-t_final': 7.000 ns,
 'id[0]-d1-carrier-freq': 5.050 GHz 2pi,
 'id[0]-d1-carrier-framechange': 5.933 rad,
 'rx90p[0]-d1-gauss-amp': 450.000 mV,
 'rx90p[0]-d1-gauss-delta': -1.000 ,
 'rx90p[0]-d1-gauss-freq_offset': -50.500 MHz 2pi,
 'rx90p[0]-d1-gauss-xy_angle': -444.089 arad,
 'rx90p[0]-d1-gauss-sigma': 1.750 ns,
 'rx90p[0]-d1-gauss-t_final': 7.000 ns,
 'rx90p[0]-d1-carrier-freq': 5.050 GHz 2pi,
 'rx90p[0]-d1-carrier-framechange': 0.000 rad,
 'ry90p[0]-d1-gauss-amp': 450.000 mV,
 'ry90p[0]-d1-gauss-delta': -1.000 ,
 'ry90p[0]-d1-gauss-freq_offset': -50.500 MHz 2pi,
 'ry90p[0]-d1-gauss-xy_angle

To access a specific parameter, e.g. the frequency of qubit 1, we use the identifying tuple `('Q1','freq')`.

In [4]:
pmap.get_parameter(('Q1','freq'))

5.000 GHz 2pi

## The opt_map
To deal with multiple parameters we use the `opt_map`, a nested list of identifyers.

In [5]:
opt_map = [
    [
        ("Q1", "freq")
    ],
    [
        ("Q1", "anhar")
    ],  
]

Here, we get a list of the parameter values:

In [6]:
pmap.get_parameters(opt_map)

[5.000 GHz 2pi, -210.000 MHz 2pi]

Let's look at the amplitude values of two gaussian control pulses, rotations about the $X$ and $Y$ axes repsectively.

In [7]:
opt_map = [
    [
        ('rx90p[0]','d1','gauss','amp')
    ],
    [
        ('ry90p[0]','d1','gauss','amp')
    ],  
]

In [8]:
pmap.get_parameters(opt_map)

[450.000 mV, 450.000 mV]

We can set the parameters to new values.

In [9]:
pmap.set_parameters([0.5, 0.6], opt_map)

In [10]:
pmap.get_parameters(opt_map)

[500.000 mV, 600.000 mV]

The opt_map also allows us to specify that two parameters should have identical values. Here, let's demand our $X$ and $Y$ rotations use the same amplitude.

In [11]:
opt_map_ident = [
    [
        ('rx90p[0]','d1','gauss','amp'),
        ('ry90p[0]','d1','gauss','amp')
    ],
]

The grouping here means that these parameters share their numerical value.

In [37]:
pmap.set_parameters([0.432], opt_map_ident)
pmap.get_parameters(opt_map_ident)

[432.000 mV]

In [38]:
pmap.get_parameters(opt_map)

[432.000 mV, 432.000 mV]

During an optimization, the varied parameters do not change, so we fix the opt_map

In [39]:
pmap.set_opt_map(opt_map)

In [63]:
pmap.get_parameters()

[500.000 mV, 400.000 mV]

## Optimizer scaling
To be independent of the choice of numerical optimizer, they should use the methods

In [65]:
pmap.get_parameters_scaled()

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([ 0., -1.], dtype=float32)>

To provide values bound to $[-1, 1]$. Let's set the parameters to their allowed minimum an maximum value with

In [66]:
pmap.set_parameters_scaled([1.0,-1.0])

In [67]:
pmap.get_parameters()

[600.000 mV, 400.000 mV]

As a safeguard, when setting values outside of the unit range, their physical values get looped back in the specified limits.

In [69]:
pmap.set_parameters_scaled([2.0, 3.0])

In [70]:
pmap.get_parameters()

[500.000 mV, 400.000 mV]

## Storing and reading
For optimization purposes, we can store and load parameter values in [HJSON](https://hjson.github.io/) format.

In [72]:
pmap.store_values("current_vals.c3log")

In [73]:
!cat current_vals.c3log

{
  opt_map:
  [
    [
      rx90p[0]-d1-gauss-amp
    ]
    [
      ry90p[0]-d1-gauss-amp
    ]
  ]
  units:
  [
    V
    V
  ]
  optim_status:
  {
    params:
    [
      0.5
      0.4000000059604645
    ]
  }
}


In [74]:
pmap.load_values("current_vals.c3log")